In [1]:
from sklearn.metrics.cluster import supervised
from scipy.optimize import linear_sum_assignment

def accuracy(labels_true, labels_pred):
    labels_true, labels_pred = supervised.check_clusterings(labels_true, labels_pred)
    # labels_true : int array with ground truth labels, shape = [n_samples]
    # labels_pred : int array with estimated labels, shape = [n_samples]
    value = supervised.contingency_matrix(labels_true, labels_pred)
    # value : array of shape [n, n] whose (i, j)-th entry is the number of samples in true class i and in predicted class j
    [r, c] = linear_sum_assignment(-value)
    return value[r, c].sum() / len(labels_true)

In [2]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Flatten, Activation, add
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import initializers
from keras.engine import Layer, InputSpec
from keras import backend as K

from resnet_keras import resnet152_model

weights_path = 'resnet_data/resnet152_weights_tf.h5'

model = resnet152_model(weights_path)
sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])


layer_name = 'res5a_relu'

intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)


C:\Users\Mason Cole\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
import scipy.io

def get_labels_from_file(data_name):
    mat = scipy.io.loadmat('data/' + data_name + '-' + 'train')
    return np.squeeze(mat['labels_train'])

def get_resnet_features_from_file(data_name, train=True):
    
    mat = scipy.io.loadmat('data/' + data_name + '-' + ('train.mat' if train else 'test.mat'))
    
    images_name = 'images_' + ('train' if train else 'test')
    
    x = np.reshape(np.transpose(np.repeat(mat[images_name][:,:, np.newaxis],3, axis=2), (1, 0, 2)), (-1, 28, 28, 3))
#     x = x * 256 - 128
    
    large_images = np.array([scipy.misc.imresize(scipy.misc.imrotate(image, -90), size=(224,224), interp='bilinear') for image in x])

    
    
    intermediate_output = intermediate_layer_model.predict(large_images)
    return np.mean(intermediate_output, (1,2))

In [4]:
import sklearn.cluster
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
import numpy as np

kmeans = sklearn.cluster.Birch(n_clusters=5).fit_predict(np.squeeze(np.mean(intermediate_output, (1,2))))
accuracy(kmeans, np.reshape(mat['labels_train'],-1))

NameError: name 'intermediate_output' is not defined

In [ ]:
np.savetxt('digits.csv', kmeans + 1, fmt='%i', delimiter=',')

In [ ]:
import scipy.io
import matplotlib.pyplot as plt
import numpy as np

mat = scipy.io.loadmat('data/digits-test.mat')
feat = mat['fea_hog_test']

kmeans = sklearn.cluster.Birch(n_clusters=5, threshold=0.7).fit_predict(np.transpose(feat))
np.savetxt('digits.csv', kmeans + 1, fmt='%i', delimiter=',', newline='\n')

In [ ]:
import scipy
filename = 'objects'
clusters = sklearn.cluster.Birch(n_clusters=5, threshold=0.7, branching_factor=5000).fit_predict(get_resnet_features_from_file(filename, 'test'))

print(accuracy(get_labels_from_file(filename), clusters))
np.savetxt(filename + '.csv', clusters + 1, fmt='%i', delimiter=',', newline='\n')